# Bivariate Normal Distribution Analysis

This notebook implements and visualizes the Bivariate Normal Distribution.

In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.datasets import load_iris

# Load Iris dataset and extract petal measurements
iris = load_iris()
X = iris.data[:, 2]  # Petal length in cm
Y = iris.data[:, 3]  # Petal width in cm

# Display basic information
print("Number of samples:", len(X))
print("Petal length range: [{:.2f}, {:.2f}] cm".format(X.min(), X.max()))
print("Petal width range: [{:.2f}, {:.2f}] cm".format(Y.min(), Y.max()))
print("Correlation coefficient: {:.4f}".format(np.corrcoef(X, Y)[0, 1]))

In [ ]:
# Bivariate Normal PDF implementation from scratch
def bvn_pdf(x, y, mux, muy, sx, sy, rho):
    """
    Calculate bivariate normal PDF at point (x, y).
    Formula implemented manually without statistical libraries.
    """
    # Normalization constant
    norm = 1.0 / (2.0 * np.pi * sx * sy * np.sqrt(1.0 - rho**2))
    
    # Standardized distances from mean
    zx = (x - mux) / sx
    zy = (y - muy) / sy
    
    # Quadratic form accounting for correlation
    Q = zx**2 + zy**2 - 2.0 * rho * zx * zy
    
    # Gaussian exponent
    exponent = -Q / (2.0 * (1.0 - rho**2))
    
    return norm * np.exp(exponent)


# Calculate parameters from data
mux = np.mean(X)
muy = np.mean(Y)
sx = np.std(X, ddof=0)
sy = np.std(Y, ddof=0)
rho = np.corrcoef(X, Y)[0, 1]

print("Bivariate Normal Parameters:")
print("  mux = {:.4f}".format(mux))
print("  muy = {:.4f}".format(muy))
print("  sx = {:.4f}".format(sx))
print("  sy = {:.4f}".format(sy))
print("  rho = {:.4f}".format(rho))

# Verify: PDF at mean should be maximum
pdf_at_mean = bvn_pdf(mux, muy, mux, muy, sx, sy, rho)
print("\nPDF at mean point ({:.2f}, {:.2f}): {:.6f}".format(mux, muy, pdf_at_mean))

In [ ]:
# Calculate PDF for all data points
pdf_values = np.array([
    bvn_pdf(x, y, mux, muy, sx, sy, rho) 
    for x, y in zip(X, Y)
])

print("PDF statistics for data points:")
print("  Maximum: {:.6f}".format(pdf_values.max()))
print("  Minimum: {:.6f}".format(pdf_values.min()))
print("  Mean: {:.6f}".format(pdf_values.mean()))

# Create evaluation grid for visualization
x_min, x_max = X.min() - 0.5, X.max() + 0.5
y_min, y_max = Y.min() - 0.2, Y.max() + 0.2

x_coords = np.linspace(x_min, x_max, 100)
y_coords = np.linspace(y_min, y_max, 100)
X_grid, Y_grid = np.meshgrid(x_coords, y_coords)

# Calculate PDF on entire grid
print("\nComputing PDF on 100x100 grid...")
Z_grid = np.array([
    [bvn_pdf(x, y, mux, muy, sx, sy, rho) for x in x_coords] 
    for y in y_coords
])

print("Grid PDF range: [{:.6f}, {:.6f}]".format(Z_grid.min(), Z_grid.max()))

In [ ]:
# Create contour plot
plt.figure(figsize=(10, 8))

# Filled contours with viridis colormap
contourf = plt.contourf(X_grid, Y_grid, Z_grid, levels=15, cmap='viridis', alpha=0.7)

# Line contours with labels
contour = plt.contour(X_grid, Y_grid, Z_grid, levels=10, colors='white', linewidths=0.5)
plt.clabel(contour, inline=True, fontsize=8, fmt='%.2f')

# Overlay actual data points
plt.scatter(X, Y, c='red', s=50, alpha=0.7, edgecolors='black', linewidth=1, label='Iris data')

# Mark the mean
plt.plot(mux, muy, 'w*', markersize=20, markeredgecolor='black', markeredgewidth=2, 
         label='Mean ({:.2f}, {:.2f})'.format(mux, muy))

# Colorbar and labels
plt.colorbar(contourf, label='Probability Density f(x,y)')
plt.xlabel('Petal Length (cm)', fontsize=12)
plt.ylabel('Petal Width (cm)', fontsize=12)
plt.title('Bivariate Normal Distribution - Contour Plot\nIris Dataset: Petal Length vs Width', 
         fontsize=14)
plt.legend(loc='upper left')
plt.grid(True, alpha=0.3, linestyle='--')

# Add parameter annotation box
param_text = 'mu_x = {:.2f}\nmu_y = {:.2f}\nsigma_x = {:.2f}\nsigma_y = {:.2f}\nrho = {:.3f}'.format(mux, muy, sx, sy, rho)
plt.text(0.95, 0.05, param_text, transform=plt.gca().transAxes, fontsize=11,
         verticalalignment='bottom', horizontalalignment='right',
         bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))

plt.tight_layout()
plt.savefig('../figures/contour_plot.png', dpi=300, bbox_inches='tight')
plt.show()

print("Contour plot shows elliptical shapes due to high correlation (rho=0.96)")

In [ ]:
# Create 3D surface plot
fig = plt.figure(figsize=(12, 9))
ax = fig.add_subplot(111, projection='3d')

# Plot the PDF surface
surf = ax.plot_surface(X_grid, Y_grid, Z_grid, cmap='viridis', alpha=0.8, 
                       rstride=2, cstride=2, linewidth=0, antialiased=True)

# Project contours on bottom plane
ax.contour(X_grid, Y_grid, Z_grid, zdir='z', offset=Z_grid.min(), cmap='viridis', levels=10)

# Scatter actual data points at their PDF heights
ax.scatter(X, Y, pdf_values, c='red', s=30, alpha=0.6, label='Data points')

# Mark the peak
pdf_peak = bvn_pdf(mux, muy, mux, muy, sx, sy, rho)
ax.scatter([mux], [muy], [pdf_peak], c='white', s=100, marker='*', 
          edgecolors='black', linewidth=2, label='Peak')

# Axis labels
ax.set_xlabel('Petal Length (cm)', fontsize=11, labelpad=10)
ax.set_ylabel('Petal Width (cm)', fontsize=11, labelpad=10)
ax.set_zlabel('Probability Density\nf(x,y)', fontsize=11, labelpad=10)
ax.set_title('Bivariate Normal Distribution - 3D Surface Plot', fontsize=14)

# Viewing angle
ax.view_init(elev=30, azim=45)

# Colorbar
fig.colorbar(surf, shrink=0.5, aspect=10, label='PDF Value')

ax.legend()
plt.tight_layout()
plt.savefig('../figures/3d_surface.png', dpi=300, bbox_inches='tight')
plt.show()

print("3D plot shows sharp diagonal ridge due to strong correlation")

In [ ]:
# Verification checks
print("=" * 50)
print("VERIFICATION OF IMPLEMENTATION")
print("=" * 50)

# Check 1: Peak location
peak_idx = np.unravel_index(np.argmax(Z_grid), Z_grid.shape)
peak_x, peak_y = X_grid[peak_idx], Y_grid[peak_idx]
print("\n1. Peak Location Check:")
print("   Expected peak at mean: ({:.4f}, {:.4f})".format(mux, muy))
print("   Actual grid maximum: ({:.4f}, {:.4f})".format(peak_x, peak_y))
print("   Difference: ({:.4f}, {:.4f})".format(abs(peak_x-mux), abs(peak_y-muy)))

# Check 2: PDF normalization (volume should be ~1)
dx = x_coords[1] - x_coords[0]
dy = y_coords[1] - y_coords[0]
volume = np.sum(Z_grid) * dx * dy
print("\n2. Normalization Check:")
print("   Volume under PDF surface: {:.4f}".format(volume))
print("   Expected: 1.0 (for valid probability density function)")
if 0.95 <= volume <= 1.05:
    print("   Status: PASS - PDF is properly normalized")
else:
    print("   Status: WARNING - Volume deviates from 1.0")

# Check 3: Correlation effect
print("\n3. Correlation Effect:")
print("   Correlation rho = {:.3f} (very strong positive)".format(rho))
print("   This creates the elongated elliptical contours seen in plots")
print("   If rho=0, contours would be circular")
print("   If rho<0, ellipse would tilt opposite direction")

# Check 4: PDF values range
print("\n4. PDF Value Range:")
print("   Maximum PDF (at mean): {:.4f}".format(Z_grid.max()))
print("   Minimum PDF (at edges): {:.6f}".format(Z_grid.min()))
print("   Ratio: {:.0f}x difference between peak and edge".format(Z_grid.max()/Z_grid.min()))

print("\n" + "=" * 50)
print("All checks completed. Implementation verified.")
print("=" * 50)